In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))
import seaborn as sns
sns.set_theme(style="whitegrid")


In [ ]:
import sys
# sys.path.append('../')
sys.path.append('../')
sys.path.append('../model')
sys.path.append('../cost')
sys.path.append('../constraints')
sys.path.append('../sensor')
sys.path.append('../utils')
import QuadRotorPointMassModel
import QuadRotorPointMassCost
import QuadRotorPointMassConstraints
import QuadRotorSensor
from scipy.integrate import solve_ivp
from Scvx import Scvx
from utils_obs import generate_obstacle_random

In [ ]:
ix = 6
iu = 3
ih = 4
tf = 5
N = 50
delT = tf/N

In [ ]:
# obstacle
r_safe = 0.3
point_range = 0.2
c,H_obs,H_safe,r = generate_obstacle_random(r_safe,5)

In [ ]:
plt.figure(figsize=(5,8))
ax=plt.gca()
for ce,re in zip(c,r) :
    circle1 = plt.Circle((ce[0],ce[1]),re,color='tab:red',alpha=0.5,fill=True)
    circle2 = plt.Circle((ce[0],ce[1]),re+r_safe,linestyle='--',color='tab:red',alpha=0.5,fill=False)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
plt.axis([-2.5, 2.5, -1, 7])
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
xi = np.array([2,6,0,0,0,0])
xf = np.array([-2,0,0,0,0,0])

myModel = QuadRotorPointMassModel.quadrotorpm('Hello',ix,iu,delT,linearization="analytic")
myCost = QuadRotorPointMassCost.quadrotorpm('Hello',ix,iu,N)
myConst = QuadRotorPointMassConstraints.quadrotorpm('Hello',ix,iu,c,H_safe)
myObs = QuadRotorSensor.lidar('Hello',ix,iu)
myVicon = QuadRotorSensor.vicon('Hello',ix,iu)

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf

u0 = np.zeros((N+1,iu))
u0[:,2] = 9.81

In [ ]:
max_iter = 30
# i1 = Scvx('unicycle',N,max_iter,myModel,myCost,myConst,
#           type_discretization='zoh',w_c=1,w_vc=1e4,w_tr=1e-1)
i1 = Scvx('unicycle',N,max_iter,myModel,myCost,myConst,
          type_discretization='zoh',w_c=1,w_vc=1e4,w_tr=10,tol_vc=1e-8,tol_tr=1e-3)
x,u,xbar,ubar,total_num_iter,flag_boundary,l,l_vc,l_tr = i1.run(x0,u0,xi,xf)

In [ ]:
list_time = delT * np.array([i for i in range(N+1)])
u_norm = np.linalg.norm(u,axis=1)
gimbal_angle = np.arccos(u[:,2] / u_norm)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5,8))
ax=plt.gca()
for ce,re in zip(c,r) :
    circle1 = plt.Circle((ce[0],ce[1]),re,color='tab:red',alpha=0.5,fill=True)
    circle2 = plt.Circle((ce[0],ce[1]),re+r_safe,linestyle='--',color='tab:red',alpha=0.5,fill=False)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
plt.plot(xbar[:,0],xbar[:,1],color='tab:blue')
plt.axis([-2.5, 2.5, -1, 7])
plt.gca().set_aspect('equal', adjustable='box')
plt.plot(0,0,color='tab:blue',label="path")
plt.plot(xi[0],xi[1],'o',color='tab:green',label="start")
plt.plot(-2,0,'*',color='tab:green',label="goal")
plt.legend(fontsize=15)

In [ ]:
# %matplotlib qt
# def data_for_cylinder_along_z(center_x,center_y,radius,height_z):
#     z = np.linspace(-height_z, height_z, 50)
#     theta = np.linspace(0, 2*np.pi, 50)
#     theta_grid, z_grid=np.meshgrid(theta, z)
#     x_grid = radius*np.cos(theta_grid) + center_x
#     y_grid = radius*np.sin(theta_grid) + center_y
#     return x_grid,y_grid,z_grid
# from mpl_toolkits.mplot3d import art3d
# fig = plt.figure(figsize=(15,15))
# ax = fig.add_subplot(111, projection='3d')
# ax.set_xlabel('X, east')
# ax.set_ylabel('Y, north')
# ax.set_zlabel('Z, up')
# ax.plot(x[:,0],x[:,1],x[:,2])
# ax.set_xlim([-3,3])
# ax.set_ylim([-1,7])
# ax.set_zlim([-1,1])

# Xc,Yc,Zc = data_for_cylinder_along_z(c[0][0],c[0][1],r1,1)
# ax.plot_surface(Xc, Yc, Zc, alpha=0.5)
# Xc,Yc,Zc = data_for_cylinder_along_z(c[1][0],c[1][1],r2,1)
# ax.plot_surface(Xc, Yc, Zc, alpha=0.5)

In [ ]:
%matplotlib inline
fS = 15
plt.figure(figsize=(15,6))
plt.subplot(311)
plt.plot(list_time[:N],u[:N,0])
plt.xlabel('time (s)',fontsize=fS)
plt.ylim([-6,6])
plt.title('T1',fontsize=fS)
plt.subplot(312)
plt.plot(list_time[:N],u[:N,1])
plt.xlabel('time (s)',fontsize=fS)
plt.ylim([-6,6])
plt.title('T2',fontsize=fS)
plt.subplot(313)
plt.plot(list_time[:N],u[:N,2])
plt.xlabel('time (s)',fontsize=fS)
plt.ylim([0,20])
plt.title('T3',fontsize=fS)


plt.figure(figsize=(15,3))
plt.plot(list_time[:N],u_norm[:N])
plt.plot(list_time[:N],list_time[:N]*0+myConst.T_max,'--',color='tab:orange')
plt.plot(list_time[:N],list_time[:N]*0+myConst.T_min,'--',color='tab:orange')
plt.xlabel('time (s)',fontsize=fS)
plt.ylim([0,32])
plt.title('u_norm',fontsize=fS)

plt.figure(figsize=(15,3))
plt.plot(list_time[:N],np.rad2deg(gimbal_angle[:N]))
plt.plot(list_time[:N],list_time[:N]*0+np.rad2deg(myConst.delta_max),'--',color='tab:orange')
plt.xlabel('time (s)',fontsize=fS)
plt.title('gimbal angle',fontsize=fS)
plt.ylim([0,np.rad2deg(myConst.delta_max)+10])

In [ ]:
np.arctan2(0,0)

In [ ]:
vis = myVicon.state2obs(x[0],c,H_obs)
print(vis['length'])

In [ ]:
vis = myVicon.state2obs(x[10],c,H_obs)
print(vis['length'])

In [ ]:
idx = 20
xp = x[idx]
vis = myVicon.state2obs(x[idx],c,H_obs)
lp = np.reshape(vis['length'],(-1,3))
plt.figure(figsize=(5,8))
ax=plt.gca()
for ce,re in zip(c,r) :
    circle1 = plt.Circle((ce[0],ce[1]),re,color='tab:red',alpha=0.5,fill=True)
    circle2 = plt.Circle((ce[0],ce[1]),re+r_safe,linestyle='--',color='tab:red',alpha=0.5,fill=False)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
for ro in lp :
    if ro[2] == 100 :
        print("a")
        continue
    else :
        plt.plot([xp[0],xp[0]+ro[0]],[xp[1],xp[1]+ro[1]],color='tab:blue')
        break
#     plt.plot(ro[0],ro[1],'o',color='tab:blue')
plt.plot(xp[0],xp[1],'o',color='black')
plt.axis([-2.5, 2.5, -1, 7])



plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
# myObs = QuadRotorSensor.lidar('Hello',ix,iu)
# vis = myObs.state2obs(x[25],c,H_obs,method=3)
# print(vis['length'])
# print(vis['point'])

In [ ]:
# myObs = QuadRotorSensor.lidar('Hello',ix,iu)
# vis = myObs.state2obs(x[25],c,H_obs,method=2)
# print(vis['length'])
# print(vis['point'])

In [ ]:
obs = myObs.state2obs(x,c,H_obs,method=2)

In [ ]:
idx = 0
xp = x[idx]
lp = obs['point'][idx]
print(lp[0])
plt.figure(figsize=(5,8))
ax=plt.gca()
for ce,re in zip(c,r) :
    circle1 = plt.Circle((ce[0],ce[1]),re,color='tab:red',alpha=0.5,fill=True)
    circle2 = plt.Circle((ce[0],ce[1]),re+r_safe,linestyle='--',color='tab:red',alpha=0.5,fill=False)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
for ro in lp :
    plt.plot([xp[0],ro[0]],[xp[1],ro[1]],color='tab:green')
    plt.plot(ro[0],ro[1],'o',color='tab:green')
plt.plot(xp[0],xp[1],'o',color='black',label='vehicle')
plt.axis([-2.5, 2.5, -1, 7])
plt.xlabel('X (m)',fontsize=15)
plt.ylabel('Y (m)',fontsize=15)

# plt.plot(xi[0],xi[1],'o',color='tab:green',label="start")
# plt.plot(-2,0,'*',color='tab:green',label="goal")
# only for label
plt.plot(-200,0,'o-',color='tab:green',label="rangefinder")
circle1 = plt.Circle((-200,0),0.1,color='tab:red',alpha=0.5,fill=True,label='obstacle')
circle2 = plt.Circle((-200,0),re+r_safe,linestyle='--',color='tab:red',alpha=0.5,fill=False,label='safe boundary')
ax.add_patch(circle1)
ax.add_patch(circle2)

plt.legend(fontsize=15)
plt.gca().set_aspect('equal', adjustable='box')
# plt.savefig('/Users/keemtw/Documents/Papers/2022 ACC/lyx/images/raw_images/obstacle_environ2.pdf')

In [ ]:
import imageio
from utils import make_quadrotor_trajectory_fig
make_quadrotor_trajectory_fig(x,obs,c,H,r)